Includes:
- importing file
- reading data
- combining data for each data system
- combining two system data
- grouping data for each restaurant on every day
- excrating new data 

### Importing main libraries

In [1]:
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime,timedelta 


/Users/leilasapple/anaconda/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/Users/leilasapple/.matplotlib/matplotlibrc", line #621
  (fname, cnt))


### Defining functions

In [2]:
def make_dataset(data,df_info):
    df=data.dropna()
    df_info=df_info.dropna()
    df["genre_name"] = np.nan
    df["area_name"] = np.nan
    df["latitude"] = np.nan
    df["longitude"] = np.nan
    df["holiday"] = np.nan



    df.visit_datetime=df.visit_datetime.apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
    df.reserve_datetime=df.reserve_datetime.apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S') )

    df["time_difference"] =  (df.visit_datetime - df.reserve_datetime)/timedelta(hours=1)


    for i in range(np.shape(df_info)[0]):
        if np.isin(df_info.iloc[i,0],df.iloc[:,0]):
            store_loc=np.where(df.iloc[:,0] == str(df_info.iloc[i,0]))[0]
            df.loc[store_loc,["genre_name"]]=df_info.iloc[i,1]
            df.loc[store_loc,["area_name"]]=df_info.iloc[i,2]
            df.loc[store_loc,["latitude"]]=df_info.iloc[i,3]
            df.loc[store_loc,["longitude"]]=df_info.iloc[i,4]


    df["visit_dayofweek"] =  df.visit_datetime.apply(lambda x: datetime.weekday(x))
#    df["visit_year"] =  df.visit_datetime.apply(lambda x: x.year)
    df["visit_month"] =  df.visit_datetime.apply(lambda x: x.month)
    df["visit_day"] =  df.visit_datetime.apply(lambda x: x.day)
    df["visit_hour"] =  df.visit_datetime.apply(lambda x: x.hour)
#    df["reserve_dayofweek"] =  df.reserve_datetime.apply(lambda x: datetime.weekday(x))
#    df["reserve_year"] =  df.reserve_datetime.apply(lambda x: x.year)
#    df["reserve_month"] =  df.reserve_datetime.apply(lambda x: x.month)
#    df["reserve_day"] =  df.reserve_datetime.apply(lambda x: x.day)
#    df["reserve_hour"] =  df.reserve_datetime.apply(lambda x: x.hour)
    df["visit_date"]=df.visit_datetime.apply(lambda x: x.date())
    df["visit_year"]=df.visit_datetime.apply(lambda x: x.year())
    for i,item  in enumerate(holiday_data.calendar_date):
        if np.isin(df.visit_date,item).any():
            df.holiday[np.where(df.visit_date==item)[0]]=holiday_data.loc[i,'holiday_flg']
    df=df.dropna()
    return df



### Path directory 

In [3]:
input_data_path='/Users/leilasapple/Documents/Springboard/recruit-restaurant-visitor-forecasting/input_files/'
output_data_path='/Users/leilasapple/Documents/Springboard/recruit-restaurant-visitor-forecasting/output_files/'

### Importing data
Data have come from two major websites which provide air and hpg data sets. 

- Hot Pepper Gourmet (hpg): similar to Yelp, here users can search restaurants and also make a reservation online
- AirREGI / Restaurant Board (air): similar to Square, a reservation control and cash register system


#### To read air data

In [4]:
air_reserve=pd.read_csv(os.path.join(input_data_path,'air_reserve.csv'))
air_store_info=pd.read_csv(os.path.join(input_data_path,'air_store_info.csv'))
air_visit_data=pd.read_csv(os.path.join(input_data_path,'air_visit_data.csv'))


##### Feature description of air_ files
- air_store_id: The restaurant's id in the air system
- air_genre_name: The type of the restaurant in the air system, the genre of the restaurant
- air_area_name: The name of the area of the restaurant location
- visit_datetime: The time of the reservation
- reserve_datetime: The time the reservation was made
- reserve_visitors: The number of visitors for that reservation
- visit_date: The date for visiting the restaurant
- visitors: The number of visitors to the restaurant on the dat
- latitude: The latitude of the area
- longitude: The longitude of the area

In [5]:
print('>>>>>>air_reserve:','\n');print(air_reserve.info(),'\n')
print('>>>>>>air_store_info:','\n');print(air_store_info.info(),'\n')
print('>>>>>>air_visit_data:','\n');print(air_visit_data.info(),'\n')

>>>>>>air_reserve: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92378 entries, 0 to 92377
Data columns (total 4 columns):
air_store_id        92378 non-null object
visit_datetime      92378 non-null object
reserve_datetime    92378 non-null object
reserve_visitors    92378 non-null int64
dtypes: int64(1), object(3)
memory usage: 2.8+ MB
None 

>>>>>>air_store_info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 5 columns):
air_store_id      829 non-null object
air_genre_name    829 non-null object
air_area_name     829 non-null object
latitude          829 non-null float64
longitude         829 non-null float64
dtypes: float64(2), object(3)
memory usage: 32.5+ KB
None 

>>>>>>air_visit_data: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252108 entries, 0 to 252107
Data columns (total 3 columns):
air_store_id    252108 non-null object
visit_date      252108 non-null object
visitors        252108 non-null int64
dtypes: int64(1),

#### To read hpg data

##### Feature description of hpg_files
- hpg_store_id: The restaurant's id in the hpg system
- hpg: The type of the restaurant in the hpg system, the genre of the restaurant
- air_area_name: The name of the area of the restaurant location
- visit_datetime: The time of the reservation
- reserve_datetime: The time the reservation was made
- reserve_visitors: The number of visitors for that reservation
- visit_date: The date for visiting the restaurant
- visitors: The number of visitors to the restaurant on the dat
- latitude: The latitude of the area
- longitude: The longitude of the area

In [6]:
hpg_reserve=pd.read_csv(os.path.join(input_data_path,'hpg_reserve.csv'))
hpg_store_info=pd.read_csv(os.path.join(input_data_path,'hpg_store_info.csv'))


- hpg_store_id: the restaurant's id in the hpg system
- visit_datetime: the time of the reservation
- reserve_datetime: the time the reservation was made
- reserve_visitors: the number of visitors for that reservation


In [7]:
print('>>>>>>hpg_reserve:','\n');print(hpg_reserve.info(),'\n')
print('>>>>>>hpg_store_info:','\n');print(hpg_store_info.info(),'\n')

>>>>>>hpg_reserve: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000320 entries, 0 to 2000319
Data columns (total 4 columns):
hpg_store_id        object
visit_datetime      object
reserve_datetime    object
reserve_visitors    int64
dtypes: int64(1), object(3)
memory usage: 61.0+ MB
None 

>>>>>>hpg_store_info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4690 entries, 0 to 4689
Data columns (total 5 columns):
hpg_store_id      4690 non-null object
hpg_genre_name    4690 non-null object
hpg_area_name     4690 non-null object
latitude          4690 non-null float64
longitude         4690 non-null float64
dtypes: float64(2), object(3)
memory usage: 183.3+ KB
None 



#### To read other data
- store_id_relation: This file allows you to join select restaurants that have both the air and hpg system.
- date_info: This file gives basic information about the calendar dates in the dataset.



In [8]:
store_id_relation=pd.read_csv(os.path.join(input_data_path,'store_id_relation.csv'))
holiday_data=pd.read_csv(os.path.join(input_data_path,'date_info.csv'))

- calendar_date
- day_of_week
- holiday_flg: is the day a holiday in Japan

In [9]:
print('>>>>>>store_id_relation:','\n');print(store_id_relation.info(),'\n')
print('>>>>>>holiday_data:','\n');print(holiday_data.info(),'\n')

>>>>>>store_id_relation: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
air_store_id    150 non-null object
hpg_store_id    150 non-null object
dtypes: object(2)
memory usage: 2.4+ KB
None 

>>>>>>holiday_data: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 3 columns):
calendar_date    517 non-null object
day_of_week      517 non-null object
holiday_flg      517 non-null int64
dtypes: int64(1), object(2)
memory usage: 12.2+ KB
None 



In [10]:
holiday_data.calendar_date=holiday_data.calendar_date.apply(lambda x:pd.to_datetime(x).date())


#### Combine two data sets of air system
The data of the store like arae and genre combined with reservation data in air system. 

In [11]:
df_air=make_dataset(air_reserve,air_store_info)
df_air=df_air.rename(columns={"air_store_id": "store_id"})


/Users/leilasapple/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
df_air.describe()

,reserve_visitors,latitude,longitude,holiday,time_difference,visit_dayofweek,visit_month,visit_day,visit_hour
count,92378.000000,92378.000000,92378.000000,92378.000000,92378.000000,92378.000000,92378.000000,92378.00000,92378.000000
mean,4.481749,36.296511,137.258264,0.066228,199.097512,3.426519,5.506398,16.19641,18.052664
std,4.919669,2.994006,3.802518,0.248681,615.662749,1.813643,4.229256,8.63795,3.105425
min,1.000000,33.211967,130.348436,0.000000,0.000000,0.000000,1.000000,1.00000,0.000000
25%,2.000000,34.681261,134.841177,0.000000,5.000000,2.000000,2.000000,9.00000,18.000000
50%,3.000000,35.641463,139.036327,0.000000,50.000000,4.000000,4.000000,17.00000,19.000000
75%,5.000000,35.694003,139.770825,0.000000,189.000000,5.000000,11.000000,24.00000,20.000000
max,100.000000,44.020632,144.273398,1.000000,9435.000000,6.000000,12.000000,31.00000,23.000000


In [13]:
print(hpg_reserve.isnull().any())
print(hpg_store_info.isnull().any())

hpg_store_id        False
visit_datetime      False
reserve_datetime    False
reserve_visitors    False
dtype: bool
hpg_store_id      False
hpg_genre_name    False
hpg_area_name     False
latitude          False
longitude         False
dtype: bool


#### Combine two data sets of hpg system
The data of the store like arae and genre combined with reservation data in hpg system.

In [14]:
hpg_reserve_=hpg_reserve.dropna()
hpg_store_info=hpg_store_info.dropna()
df_hpg=make_dataset(hpg_reserve_,hpg_store_info)
df_hpg=df_hpg.rename(columns={"hpg_store_id": "store_id"})
df_hpg=df_hpg.dropna()

/Users/leilasapple/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Combine two data sets of air system and hpg system
Two data sets combined of air system and hpg stystem

In [15]:
for i, item in enumerate(store_id_relation.iloc[:,1]):
    if np.isin(df_hpg.iloc[:,0],item).any():
        hpg_loc=np.where(df_hpg.iloc[:,0]==item)[0]
        df_hpg.iloc[hpg_loc,0]=store_id_relation.iloc[i,0]
df=pd.concat([df_air,df_hpg],ignore_index=True)


       reserve_visitors       latitude      longitude        holiday  \
count     963686.000000  963686.000000  963686.000000  963686.000000   
mean           5.253537      35.901246     137.469843       0.070904   
std            5.810087       2.349505       3.354441       0.256664   
min            1.000000      33.211967     130.339313       0.000000   
25%            2.000000      34.692109     135.483426       0.000000   
50%            3.000000      35.646745     139.373870       0.000000   
75%            6.000000      35.704527     139.737998       0.000000   
max          100.000000      44.020632     144.273398       1.000000   

       time_difference  visit_dayofweek    visit_month      visit_day  \
count    963686.000000    963686.000000  963686.000000  963686.000000   
mean        176.363289         3.529762       5.917852      16.684587   
std         276.287772         1.765237       3.914088       8.577741   
min           0.000000         0.000000       1.000000     

In [26]:
df["visit_year"]=df.visit_datetime.apply(lambda x: x.year)

In [27]:
print(df.describe())
print(df.head())

       reserve_visitors       latitude      longitude        holiday  \
count     963686.000000  963686.000000  963686.000000  963686.000000   
mean           5.253537      35.901246     137.469843       0.070904   
std            5.810087       2.349505       3.354441       0.256664   
min            1.000000      33.211967     130.339313       0.000000   
25%            2.000000      34.692109     135.483426       0.000000   
50%            3.000000      35.646745     139.373870       0.000000   
75%            6.000000      35.704527     139.737998       0.000000   
max          100.000000      44.020632     144.273398       1.000000   

       time_difference  visit_dayofweek    visit_month      visit_day  \
count    963686.000000    963686.000000  963686.000000  963686.000000   
mean        176.363289         3.529762       5.917852      16.684587   
std         276.287772         1.765237       3.914088       8.577741   
min           0.000000         0.000000       1.000000     

To backup all data in a csv file.

In [29]:
df.to_csv(os.path.join(output_data_path,'df0.csv'), encoding='utf-8',index=False)

### Data for daily resrevation of each stores

All recoreds are for each reservation made online. So there are several reservation every day for each restaurant. To have data only for each day, the number of visitors added up. 
- We need to know how many customer went to each resturant


In [31]:
df['store_id_date']=df.store_id.values+'_'+df.visit_date.astype(str).values
uniq_store_date=df['store_id_date'].unique().astype(str)

In [32]:
#data=pd.read_csv(os.path.join(output_data_path,'data0.csv'))
data=pd.DataFrame(data=uniq_store_date,columns=['id'])

Grouping data by resturant and day 
data were grouped by the resurant and the day of visit. So every resturant-day is a group. Data were grouped by resurant id store and the day

In [33]:
grouped=df.groupby('store_id_date')

- Gener, area, holiday, day and month in every group are the same based on the grouping 
- Visitors in every group are summation of all visitors in each reservation for that group
- The time differnce for each group is taken as the average of time difference of all reservations in a that group 


In [39]:
grouped.get_group(df.loc[1,'store_id_date']).keys()


Index(['store_id', 'visit_datetime', 'reserve_datetime', 'reserve_visitors',
       'genre_name', 'area_name', 'latitude', 'longitude', 'holiday',
       'time_difference', 'visit_dayofweek', 'visit_month', 'visit_day',
       'visit_hour', 'visit_date', 'store_id_date', 'visit_year'],
      dtype='object')

In [40]:
data['store_id']=data.id.apply(lambda x: x[:-11])
data['genre']=data.id.apply(lambda x: grouped.get_group(x).iloc[0,4])
data['visitor']=data.id.apply(lambda x: grouped.get_group(x).iloc[:,3].sum())
data['area']=data.id.apply(lambda x: grouped.get_group(x).iloc[0,5])
data['holiday']=data.id.apply(lambda x: grouped.get_group(x).iloc[0,8])
data['dt']=data.id.apply(lambda x: grouped.get_group(x).iloc[:,9].mean())
data['dayofweek']=data.id.apply(lambda x: grouped.get_group(x).iloc[0,10])
data['month']=data.id.apply(lambda x: grouped.get_group(x).iloc[0,11])
data['day']=data.id.apply(lambda x: grouped.get_group(x).iloc[0,12])
data['year']=data.id.apply(lambda x: grouped.get_group(x).iloc[0,16])
data['date']=data.id.apply(lambda x: grouped.get_group(x).iloc[0,14])


In [43]:
data['hour']=data.id.apply(lambda x: grouped.get_group(x).iloc[:,13].mean())


In [6]:
# backup the data
data.to_csv(os.path.join(output_data_path,'data.csv'), encoding='utf-8',index=False)
data=pd.read_csv(os.path.join(output_data_path,'data.csv'))


In [9]:
data=pd.read_csv(os.path.join(output_data_path,'data.csv'))
print(data.head())
print(data.describe())

                                id              store_id          genre  \
0  air_877f79706adbfb06_2016-01-01  air_877f79706adbfb06  Japanese food   
1  air_db4b38ebe7a7ceff_2016-01-01  air_db4b38ebe7a7ceff     Dining bar   
2  air_db80363d35f10926_2016-01-01  air_db80363d35f10926     Dining bar   
3  air_db80363d35f10926_2016-01-02  air_db80363d35f10926     Dining bar   
4  air_3bb99a1fe0583897_2016-01-02  air_3bb99a1fe0583897        Izakaya   

   visitor                             area  holiday    dt  dayofweek  month  \
0        3     Tōkyō-to Minato-ku Shibakōen      1.0   3.5          4      1   
1        9     Ōsaka-fu Ōsaka-shi Shinmachi      1.0   0.0          4      1   
2        9  Hokkaidō Asahikawa-shi 6 Jōdōri      1.0  11.5          4      1   
3       36  Hokkaidō Asahikawa-shi 6 Jōdōri      1.0   7.3          5      1   
4        4   Miyagi-ken Sendai-shi Kamisugi      1.0  11.0          5      1   

   day  year        date  hour  
0    1  2016  2016-01-01  19.5  
1 

### Extracting new data 
Generate new set of data for the model
This data set has unique values for each resturant in every day.
The data were grouped by day,month, hoiday , area and genre and then the average of the visitors calculated for each restaurant every day.

* grouping data by the resturant only

In [10]:
grouped=data.groupby('store_id')
data['store_index']=np.nan
for i,item in enumerate(grouped.groups):
    loc_item=np.where(data['store_id']==item)[0]
    data.loc[loc_item,'store_index']=grouped.get_group(item).loc[:,'visitor'].mean()

* To check the effect of location on reservations 

In [11]:
grouped=data.groupby('area')
data['area_index']=np.nan
for i,item in enumerate(grouped.groups):
    loc_item=np.where(data['area']==item)[0]
    data.loc[loc_item,'area_index']=grouped.get_group(item).loc[:,'visitor'].mean()

* To check the effect of genre on reservations

In [12]:
grouped=data.groupby('genre')
data['genre_index']=np.nan
for i,item in enumerate(grouped.groups):
    loc_item=np.where(data['genre']==item)[0]
    data.loc[loc_item,'genre_index']=grouped.get_group(item).loc[:,'visitor'].mean()

* To check the effect of both genre and location on reservations   


In [13]:
grouped=data.groupby(['genre','area'])
data['genre_area_index']=np.nan
for i,item in enumerate(grouped.groups):
    a=data.genre==item[0]
    b=data.area==item[1]
    LOC=np.where(a & b)[0]
    data.loc[LOC,'genre_area_index']=grouped.get_group(item).loc[:,'visitor'].mean()

* To check the effect of the holiday on reservation 

In [14]:
grouped=data.groupby('holiday')
data['holiday_index']=np.nan
for i,item in enumerate(grouped.groups):
    loc_item=np.where(data['holiday']==item)[0]
    data.loc[loc_item,'holiday_index']=grouped.get_group(item).loc[:,'visitor'].mean()

* To check the effect of the holiday for each resturant on reservation 

In [15]:
grouped=data.groupby(['store_id','holiday'])
data['store_holiday_index']=np.nan
for i,item in enumerate(grouped.groups):
    a=data.store_id==item[0]
    b=data.holiday==item[1]
    LOC=np.where(a & b)[0]
    data.loc[LOC,'store_holiday_index']=grouped.get_group(item).loc[:,'visitor'].mean()

* To check the effect of the month on reservation 

In [16]:
grouped=data.groupby('month')
data['month_index']=np.nan
for i,item in enumerate(grouped.groups):
    loc_item=np.where(data['month']==item)[0]
    data.loc[loc_item,'month_index']=grouped.get_group(item).loc[:,'visitor'].mean()

* To check the effect of the month for each resturant on reservation

In [17]:
grouped=data.groupby(['store_id','month'])
data['store_month_index']=np.nan
for i,item in enumerate(grouped.groups):
    a=data.store_id==item[0]
    b=data.month==item[1]
    LOC=np.where(a & b)[0]
    data.loc[LOC,'store_month_index']=grouped.get_group(item).loc[:,'visitor'].mean()

* To check the effect of the day of the week on reservation

In [18]:
grouped=data.groupby('dayofweek')
data['dayofweek_index']=np.nan
for i,item in enumerate(grouped.groups):
    loc_item=np.where(data['dayofweek']==item)[0]
    data.loc[loc_item,'dayofweek_index']=grouped.get_group(item).loc[:,'visitor'].mean()

* To check the effect of the day of the week for each resturant on reservation

In [19]:
grouped=data.groupby(['store_id','dayofweek'])
data['store_dayofweek_index']=np.nan
for i,item in enumerate(grouped.groups):
    a=data.store_id==item[0]
    b=data.dayofweek==item[1]
    LOC=np.where(a & b)[0]
    if i%5000==0:print(i,item,datetime.now())
    data.loc[LOC,'store_dayofweek_index']=grouped.get_group(item).loc[:,'visitor'].mean()

0 ('air_00a91d42b08b08d9', 0) 2018-04-28 08:35:01.117115
5000 ('hpg_19bd36fe318f4f4a', 5) 2018-04-28 08:40:24.212479
10000 ('hpg_413ab82a50eca566', 2) 2018-04-28 08:45:47.678115
15000 ('hpg_6c926abc4ebaff35', 6) 2018-04-28 08:51:10.345585
20000 ('hpg_94d65882a11d596c', 6) 2018-04-28 08:56:34.066907
25000 ('hpg_bc33446f4f01112f', 0) 2018-04-28 09:01:58.769326
30000 ('hpg_e35b6421237507e0', 1) 2018-04-28 09:07:23.680920


* To check the effect of the day of the week in each month on reservation

In [20]:
grouped=data.groupby(['dayofweek','month'])
data['dayofweek_month_index']=np.nan
for i,item in enumerate(grouped.groups):
    a=data.dayofweek==item[0]
    b=data.month==item[1]
    LOC=np.where(a & b)[0]
    data.loc[LOC,'dayofweek_month_index']=grouped.get_group(item).loc[:,'visitor'].mean()

* To check the effect of the day of the week and the month for each resturant on reservation

In [21]:
grouped=data.groupby(['store_id','dayofweek','month'])
data['store_dayofweek_month_index']=np.nan
for i,item in enumerate(grouped.groups):
    a=data.store_id==item[0]
    b=data.dayofweek==item[1]
    c=data.month==item[2]
    LOC=np.where(a & b & c)[0]
    if i%100 ==0: print(i,item)
    data.loc[LOC,'store_dayofweek_month_index']=grouped.get_group(item).loc[:,'visitor'].mean()

0 ('air_00a91d42b08b08d9', 0, 10)
100 ('air_03963426c9312048', 5, 3)
200 ('air_066f0221b8a4d533', 1, 12)
300 ('air_0867f7bebad6a649', 4, 4)
400 ('air_08ef81d5b7a0d13f', 0, 12)
500 ('air_0a74a5408a0b8642', 6, 7)
600 ('air_0e1eae99b8723bc1', 2, 11)
700 ('air_1033310359ceeac1', 2, 8)
800 ('air_138ff410757b845f', 1, 12)
900 ('air_142e78ba7001da9c', 4, 2)
1000 ('air_1707a3f18bb0da07', 5, 5)
1100 ('air_1d1e8860ae04f8e9', 5, 6)
1200 ('air_1f34e9beded2231a', 4, 10)
1300 ('air_24b9b2a020826ede', 0, 2)
1400 ('air_258ad2619d7bff9a', 4, 5)
1500 ('air_28064154614b2e6c', 2, 4)
1600 ('air_290e7a57b390f78e', 5, 4)
1700 ('air_2a485b92210c98b5', 4, 9)
1800 ('air_2b8b29ddfd35018e', 0, 4)
1900 ('air_2c6fef1ce0e13a5a', 6, 1)
2000 ('air_2d3afcb91762fe01', 2, 3)
2100 ('air_32460819c7600037', 2, 5)
2200 ('air_324f7c39a8410e7c', 6, 11)
2300 ('air_33b01025210d6007', 1, 12)
2400 ('air_346ade7d29230634', 5, 10)
2500 ('air_37189c92b6c761ec', 4, 7)
2600 ('air_396166d47733d5c9', 4, 2)
2700 ('air_399904bdb7685ca0', 3

22300 ('hpg_0b047ba98db48c12', 6, 4)
22400 ('hpg_0b4acef80c58435f', 5, 4)
22500 ('hpg_0b70de808b55ad1e', 1, 7)
22600 ('hpg_0b9fa10f5047a36e', 2, 10)
22700 ('hpg_0bb83830c9d89447', 2, 12)
22800 ('hpg_0bc482b0055f6035', 4, 11)
22900 ('hpg_0bd908f7f603db91', 3, 4)
23000 ('hpg_0c218bade8cc51b3', 4, 12)
23100 ('hpg_0c3c9ba0341870dd', 4, 7)
23200 ('hpg_0c4967900ee99b3f', 5, 6)
23300 ('hpg_0c683ce6776e0a5a', 2, 9)
23400 ('hpg_0cb3c2c490020a29', 6, 9)
23500 ('hpg_0cbbde8c5c7ea6f3', 5, 12)
23600 ('hpg_0cd0129a56af6039', 2, 1)
23700 ('hpg_0cd389cd4526b276', 5, 9)
23800 ('hpg_0d0f0a54b15904e1', 1, 7)
23900 ('hpg_0d1990d659c2045f', 5, 6)
24000 ('hpg_0d3016938cc35ac9', 6, 1)
24100 ('hpg_0d4c267571f19abe', 4, 12)
24200 ('hpg_0d66eee80800df15', 5, 1)
24300 ('hpg_0d80e2e3ddb9af45', 5, 7)
24400 ('hpg_0db71b7dd0f38710', 4, 11)
24500 ('hpg_0dcce9502b50d801', 5, 10)
24600 ('hpg_0de86eea5240a0d7', 6, 5)
24700 ('hpg_0e039102142eb41a', 3, 2)
24800 ('hpg_0e6a1de1b6fad888', 0, 2)
24900 ('hpg_0e95b25111b52b7c',

44300 ('hpg_22df88b7616c11b0', 6, 7)
44400 ('hpg_22f2333b67eed586', 1, 11)
44500 ('hpg_22f2523f72934117', 6, 2)
44600 ('hpg_2307cfff77758975', 2, 8)
44700 ('hpg_230ae7367a82e06c', 5, 9)
44800 ('hpg_231daf071676f919', 0, 4)
44900 ('hpg_2334b44de2d44587', 6, 6)
45000 ('hpg_2355da20f2938b02', 2, 12)
45100 ('hpg_23711e7cfa1f113f', 6, 5)
45200 ('hpg_238569783d34850a', 3, 2)
45300 ('hpg_238f31ddb6bac1e9', 1, 9)
45400 ('hpg_2392fa48801ad133', 6, 8)
45500 ('hpg_23a565a2f973ec8e', 3, 10)
45600 ('hpg_23be7141734c628a', 1, 2)
45700 ('hpg_23db9baac8e244c6', 0, 1)
45800 ('hpg_2403269c7e9d5015', 2, 7)
45900 ('hpg_240c94892fc9c2a9', 2, 3)
46000 ('hpg_2437c5dcb6c32841', 5, 2)
46100 ('hpg_243d50a8fecc8b48', 2, 11)
46200 ('hpg_244e11105fd1d5a3', 3, 5)
46300 ('hpg_2466d42830c1a1ec', 1, 3)
46400 ('hpg_249885db36392288', 0, 2)
46500 ('hpg_249d67c9e58f1a08', 3, 3)
46600 ('hpg_24a0b587c5113c8b', 6, 2)
46700 ('hpg_24a9c7a70c705d83', 2, 6)
46800 ('hpg_24ab4675725d7edf', 6, 8)
46900 ('hpg_24c62b742c418690', 5, 

66300 ('hpg_38f70a37dab98795', 4, 7)
66400 ('hpg_38ff17cfd3da9ffb', 2, 6)
66500 ('hpg_3900dc387c7a0103', 6, 9)
66600 ('hpg_3932d5ffb46fd226', 5, 6)
66700 ('hpg_3981b1cdc948b0a7', 2, 6)
66800 ('hpg_399a3fb840f18f2d', 2, 11)
66900 ('hpg_39c589ecfc5a8c8c', 5, 12)
67000 ('hpg_39e2e51816ef3be1', 6, 10)
67100 ('hpg_3a08e8ff5813a233', 4, 4)
67200 ('hpg_3a157c1e6dab2be3', 3, 11)
67300 ('hpg_3a1de99f7b8dd6cd', 6, 12)
67400 ('hpg_3a3289a9c062f404', 1, 3)
67500 ('hpg_3a3cd6ec19996677', 1, 3)
67600 ('hpg_3a413ab66ce2273d', 4, 2)
67700 ('hpg_3a633f8a19c7697c', 3, 12)
67800 ('hpg_3a8eced4c7bbedb9', 4, 4)
67900 ('hpg_3ab17f2bf02415f5', 5, 3)
68000 ('hpg_3ac4371082d7f7df', 0, 1)
68100 ('hpg_3aedf437feb40b1c', 2, 8)
68200 ('hpg_3af8add179100bb8', 5, 9)
68300 ('hpg_3b235216f63d0688', 1, 2)
68400 ('hpg_3b25138182a9f8f7', 6, 11)
68500 ('hpg_3b2d6557d5d580f0', 6, 10)
68600 ('hpg_3b6182bd49b83c15', 4, 5)
68700 ('hpg_3b83d0400b9192b5', 5, 7)
68800 ('hpg_3b87f4dce392ec74', 3, 4)
68900 ('hpg_3b88bda61a3f7058',

88300 ('hpg_52aaec1302846a20', 0, 8)
88400 ('hpg_52ac4a81c1dd32c4', 4, 6)
88500 ('hpg_52f6bbc5488b9b8d', 5, 9)
88600 ('hpg_52fadbd7c2092581', 5, 2)
88700 ('hpg_531ae40d79e9e4f3', 4, 5)
88800 ('hpg_532d48dd2bc81314', 4, 7)
88900 ('hpg_532f37a90bd7c13d', 4, 12)
89000 ('hpg_535044ab35e28bc0', 2, 1)
89100 ('hpg_536645da2ca96268', 4, 11)
89200 ('hpg_537e5ce5379247eb', 4, 3)
89300 ('hpg_538f182b1bd2e66a', 5, 11)
89400 ('hpg_53e7cb66e2a8dd40', 3, 3)
89500 ('hpg_53f326f98d880ec5', 6, 2)
89600 ('hpg_542ea9763f0f9c8f', 5, 2)
89700 ('hpg_5442e688878414f5', 5, 10)
89800 ('hpg_54486682b24aec14', 5, 4)
89900 ('hpg_54539ef5c72cba1e', 2, 2)
90000 ('hpg_54870e402d1c133f', 2, 4)
90100 ('hpg_54a89862d95d7e9b', 4, 6)
90200 ('hpg_54b5a6ffd96e80d8', 1, 12)
90300 ('hpg_54d3a3f3f5ccb2e7', 4, 7)
90400 ('hpg_54dd9ff429847a02', 6, 4)
90500 ('hpg_54f9fbc3002f3627', 3, 9)
90600 ('hpg_55037cef94871c1b', 2, 3)
90700 ('hpg_5533debc2794e418', 4, 12)
90800 ('hpg_5557c8cd8babde1c', 0, 3)
90900 ('hpg_55629e3b63f4ae74', 1

...

153000 ('hpg_98f4d56e18d556b4', 1, 8)
153100 ('hpg_99011664f1b9fb6e', 6, 9)
153200 ('hpg_99342b8f89a54e98', 4, 10)
153300 ('hpg_99371a66bc4c8896', 4, 4)
153400 ('hpg_997b14588fd06b27', 3, 3)
153500 ('hpg_999cda374c60a6f4', 5, 10)
153600 ('hpg_99a76133746bd8e3', 5, 8)
153700 ('hpg_99c2cc5ba9dd5a42', 3, 10)
153800 ('hpg_9a06f60aee9fa3ad', 4, 5)
153900 ('hpg_9a14281a4bc38e49', 1, 1)
154000 ('hpg_9a2b96907a52e4ff', 5, 1)
154100 ('hpg_9a54e7f1109e8503', 5, 4)
154200 ('hpg_9a70fe70b7142d3b', 4, 12)
154300 ('hpg_9a81c7f1f5c52ef7', 2, 2)
154400 ('hpg_9a837c9e0fbe8785', 4, 6)
154500 ('hpg_9a961831dbf6578d', 3, 6)
154600 ('hpg_9abed990fb16ce5d', 6, 3)
154700 ('hpg_9acb2c3f5b0a1737', 5, 1)
154800 ('hpg_9ad6c4b575e68b0b', 6, 7)
154900 ('hpg_9ae6d31cc913165b', 3, 2)
155000 ('hpg_9b0b96610900cbf5', 0, 9)
155100 ('hpg_9b1c994626528147', 4, 11)
155200 ('hpg_9b42ee2cbce0c841', 2, 3)
155300 ('hpg_9b5364ac2a164606', 4, 5)
155400 ('hpg_9b5c24a1abed3d25', 6, 2)
155500 ('hpg_9b99c231444438c5', 4, 4)
155600 

174400 ('hpg_b0a7ecaef664d7d2', 5, 1)
174500 ('hpg_b0bee008de149481', 1, 3)
174600 ('hpg_b0d9d5529eb2e55e', 1, 12)
174700 ('hpg_b0f28f2c28039f88', 2, 6)
174800 ('hpg_b11e07da82cefb53', 6, 9)
174900 ('hpg_b1607f7afdd0c67a', 0, 10)
175000 ('hpg_b1758d3797d483ea', 5, 6)
175100 ('hpg_b17afe1a30bcd4cc', 3, 5)
175200 ('hpg_b197ba6156ecddca', 0, 4)
175300 ('hpg_b19c264d78431857', 6, 8)
175400 ('hpg_b1b06a7248c71ad8', 5, 4)
175500 ('hpg_b1dd27ee9302dcc3', 0, 3)
175600 ('hpg_b1e69079b215d229', 4, 2)
175700 ('hpg_b1f61fc17bfa18fe', 2, 4)
175800 ('hpg_b20da6787a240e84', 0, 1)
175900 ('hpg_b2131e9a72e9b90e', 4, 11)
176000 ('hpg_b2289aace72dbf6a', 5, 1)
176100 ('hpg_b24d8e8bfa104549', 5, 10)
176200 ('hpg_b25861fc1e5315e1', 4, 3)
176300 ('hpg_b2721be55e497131', 2, 3)
176400 ('hpg_b281be42bb8c9dd2', 4, 2)
176500 ('hpg_b2ba03663f7e8242', 4, 12)
176600 ('hpg_b2cb791ffca91114', 3, 3)
176700 ('hpg_b2daddd9943309b2', 3, 4)
176800 ('hpg_b2dc09b3b2ca587d', 5, 1)
176900 ('hpg_b2f1f6af37877154', 0, 9)
177000 

195900 ('hpg_c4ef231b0aafa639', 3, 10)
196000 ('hpg_c4f33587014dd154', 6, 1)
196100 ('hpg_c53dd1a65de71e47', 2, 2)
196200 ('hpg_c542609d73c9a337', 1, 4)
196300 ('hpg_c552b037db671d24', 0, 11)
196400 ('hpg_c56f60f7006ac14c', 3, 11)
196500 ('hpg_c5899d3a5099c1bb', 4, 2)
196600 ('hpg_c590e1031e2638fb', 0, 2)
196700 ('hpg_c5bd86af2ece2705', 1, 4)
196800 ('hpg_c5c146e2eb839e28', 3, 11)
196900 ('hpg_c5cd05c45f07d753', 6, 3)
197000 ('hpg_c6087bdff618e2e1', 5, 6)
197100 ('hpg_c61c8179933d221b', 2, 9)
197200 ('hpg_c6355f651b0c7d0e', 5, 3)
197300 ('hpg_c641fee581eb9835', 0, 1)
197400 ('hpg_c64dff053ed28da3', 3, 4)
197500 ('hpg_c66111c11ba6382e', 5, 5)
197600 ('hpg_c684b31af7fe49c9', 1, 6)
197700 ('hpg_c684baa3ea96a840', 3, 12)
197800 ('hpg_c68c4637b740b1ce', 1, 3)
197900 ('hpg_c693c9b531f47f90', 5, 7)
198000 ('hpg_c6a925856c9dab6a', 4, 5)
198100 ('hpg_c6c5bbb13b285648', 2, 9)
198200 ('hpg_c6dc223e79a86995', 4, 10)
198300 ('hpg_c6ed353a48c7c93e', 5, 5)
198400 ('hpg_c6eea0e733d3914e', 2, 4)
198500

217300 ('hpg_dacb0be112058dda', 1, 9)
217400 ('hpg_dad7855e06b14f4b', 1, 1)
217500 ('hpg_daeb1f59e784fdc0', 3, 11)
217600 ('hpg_db0d375e7dc5b0a6', 3, 1)
217700 ('hpg_db3930bd5e1f3905', 4, 9)
217800 ('hpg_db47c13307a26561', 2, 5)
217900 ('hpg_db4a4341260a4e50', 5, 12)
218000 ('hpg_db7ac93a77091759', 3, 10)
218100 ('hpg_dbcbc5e69238a58c', 0, 3)
218200 ('hpg_dbd3900bffa1927e', 3, 1)
218300 ('hpg_dbd5887aeb9ae7b0', 2, 8)
218400 ('hpg_dbe4cf20470a39ff', 4, 6)
218500 ('hpg_dc0bbaa8af3e2e12', 4, 5)
218600 ('hpg_dc2e1a0a1a8050f0', 1, 5)
218700 ('hpg_dc4e9426d21c1d13', 3, 3)
218800 ('hpg_dc72b1de74757ef8', 0, 12)
218900 ('hpg_dc83456521b8f3c9', 0, 2)
219000 ('hpg_dc848323b917b8e6', 4, 2)
219100 ('hpg_dc9a9386786ca756', 3, 11)
219200 ('hpg_dcafe53112818dd8', 2, 11)
219300 ('hpg_dcc4828494265287', 5, 10)
219400 ('hpg_dccf0b318e23bce0', 4, 12)
219500 ('hpg_dcfe49e0106c5a85', 6, 1)
219600 ('hpg_dd2f97d75ea6474e', 2, 2)
219700 ('hpg_dd56ab5691ec4970', 6, 4)
219800 ('hpg_dd8599508288a2f7', 4, 7)
2199

238800 ('hpg_f101a770e169705b', 2, 11)
238900 ('hpg_f1084c266cd1db4c', 4, 4)
239000 ('hpg_f10a4b776ee02e77', 6, 2)
239100 ('hpg_f10d04344ebc7905', 2, 12)
239200 ('hpg_f115240564b58854', 1, 12)
239300 ('hpg_f13035ddab35b836', 0, 4)
239400 ('hpg_f1474ad66aebf858', 3, 4)
239500 ('hpg_f154291f356ef469', 4, 2)
239600 ('hpg_f193a9570ecd6f0e', 2, 1)
239700 ('hpg_f1964333c614ff10', 6, 2)
239800 ('hpg_f1abe5c87534b5bb', 4, 3)
239900 ('hpg_f1ba70f16e2152e5', 2, 3)
240000 ('hpg_f1ca36d352e6fa8d', 5, 3)
240100 ('hpg_f1f0aa7e917e2234', 4, 8)
240200 ('hpg_f200a457478b40c0', 2, 4)
240300 ('hpg_f209b2452acb1bc0', 4, 5)
240400 ('hpg_f23b00c038f70720', 3, 7)
240500 ('hpg_f2493ece76890a10', 5, 5)
240600 ('hpg_f267d081d2319232', 1, 12)
240700 ('hpg_f2700fa26895b6b7', 5, 2)
240800 ('hpg_f27e35adcf706ffb', 0, 7)
240900 ('hpg_f2a04b71832d898e', 4, 3)
241000 ('hpg_f30adae73356935a', 4, 9)
241100 ('hpg_f3101f2b95e2540a', 1, 3)
241200 ('hpg_f31869d049d254dc', 4, 8)
241300 ('hpg_f348bf541fca0c36', 0, 7)
241400 (

* To check the effect of the day of the week, the month and holiday for each resturant on reservation

In [22]:
grouped=data.groupby(['holiday','dayofweek','month'])
data['holiday_dayofweek_month_index']=np.nan
for i,item in enumerate(grouped.groups):
    
    a=data.holiday==item[0]
    b=data.dayofweek==item[1]
    c=data.month==item[2]
    LOC=np.where(a & b & c)[0]
    data.loc[LOC,'holiday_dayofweek_month_index']=grouped.get_group(item).loc[:,'visitor'].mean()

In [23]:
print(data.describe())
print(data.head())

data.to_csv(os.path.join(output_data_path,'data1.csv'), encoding='utf-8',index=False)
data=pd.read_csv(os.path.join(output_data_path,'data1.csv'))

             visitor        holiday             dt      dayofweek  \
count  587957.000000  587957.000000  587957.000000  587957.000000   
mean        8.610766       0.064949     167.435631       3.361385   
std        10.973159       0.246435     190.366523       1.836051   
min         1.000000       0.000000       0.000000       0.000000   
25%         3.000000       0.000000      46.000000       2.000000   
50%         5.000000       0.000000     107.000000       4.000000   
75%        10.000000       0.000000     219.500000       5.000000   
max      2241.000000       1.000000    8429.284314       6.000000   

               month            day           year           hour  \
count  587957.000000  587957.000000  587957.000000  587957.000000   
mean        5.728307      16.341962    2016.344134      18.146951   
std         3.728686       8.653998       0.475085       2.135966   
min         1.000000       1.000000    2016.000000       0.000000   
25%         3.000000       9.0000

* To check the effect of the day on reservation

In [24]:
grouped=data.groupby('day')
data['day_index']=np.nan
for i,item in enumerate(grouped.groups):
    loc_item=np.where(data['day']==item)[0]
    data.loc[loc_item,'day_index']=grouped.get_group(item).loc[:,'visitor'].mean()

* To check the effect of the day for each resturant on reservation

In [25]:
grouped=data.groupby(['store_id','day'])
data['store_day_index']=np.nan
for i,item in enumerate(grouped.groups):
    a=data.store_id==item[0]
    b=data.day==item[1]
    LOC=np.where(a & b)[0]
    if i%5000==0:print(i,item,datetime.now())
    data.loc[LOC,'store_day_index']=grouped.get_group(item).loc[:,'visitor'].mean()

0 ('air_00a91d42b08b08d9', 1) 2018-04-28 13:46:20.872616
5000 ('air_a24bf50c3e90d583', 6) 2018-04-28 13:51:31.273202
10000 ('hpg_04db70a65270f6bb', 10) 2018-04-28 13:56:41.835872
15000 ('hpg_0f74472035b1b6c6', 30) 2018-04-28 14:01:53.689795
20000 ('hpg_186b31414d9c34b4', 17) 2018-04-28 14:07:04.744845
25000 ('hpg_22b14cf76730831f', 9) 2018-04-28 14:12:16.542845
30000 ('hpg_2b500fcd37208d65', 31) 2018-04-28 14:17:27.885713
35000 ('hpg_34b191f390a530a6', 27) 2018-04-28 14:22:39.027533
40000 ('hpg_3de0c96c8c83c3c9', 18) 2018-04-28 14:27:50.193310
45000 ('hpg_4905745d9d46ed43', 1) 2018-04-28 14:33:09.288637
50000 ('hpg_535da17b4fda8227', 3) 2018-04-28 14:38:44.448543
55000 ('hpg_5d55389750a431e7', 6) 2018-04-28 14:44:07.032110
60000 ('hpg_67d55f1473c34e6d', 23) 2018-04-28 14:49:04.892055
65000 ('hpg_7192b1494ff64dc8', 24) 2018-04-28 14:54:50.049690
70000 ('hpg_7a3b06f0f4731d92', 5) 2018-04-28 15:01:10.251805
75000 ('hpg_84ec373e3183da86', 1) 2018-04-28 15:07:26.837662
80000 ('hpg_8e46f7f90

* To check the effect of the day in each month on reservation

In [26]:
grouped=data.groupby(['day','month'])
data['day_month_index']=np.nan
for i,item in enumerate(grouped.groups):
    a=data.day==item[0]
    b=data.month==item[1]
    LOC=np.where(a & b)[0]
    data.loc[LOC,'day_month_index']=grouped.get_group(item).loc[:,'visitor'].mean()

* To check the effect of the day and the month on reservation

In [36]:
grouped=data.groupby(['store_id','day','month'])
data['store_day_month_index']=np.nan
for i,item in enumerate(grouped.groups):
    a=data.store_id==item[0]
    b=data.day==item[1]
    c=data.month==item[2]
    LOC=np.where(a & b & c)[0]
    if i%100 ==0: print(i,item)
    data.loc[LOC,'store_day_month_index']=grouped.get_group(item).loc[:,'visitor'].mean()

0 ('air_00a91d42b08b08d9', 1, 3)
100 ('air_034a3d5b40d5b1b1', 5, 11)
200 ('air_03963426c9312048', 15, 1)
300 ('air_03963426c9312048', 31, 10)
400 ('air_04cae7c1bc9b2a0b', 12, 10)
500 ('air_04cae7c1bc9b2a0b', 25, 6)
600 ('air_066f0221b8a4d533', 30, 11)
700 ('air_0867f7bebad6a649', 5, 3)
800 ('air_0867f7bebad6a649', 27, 3)
900 ('air_08cb3c4ee6cd6a22', 11, 6)
1000 ('air_08cb3c4ee6cd6a22', 24, 2)
1100 ('air_08ef81d5b7a0d13f', 20, 12)
1200 ('air_0a74a5408a0b8642', 10, 5)
1300 ('air_0a74a5408a0b8642', 22, 3)
1400 ('air_0b184ec04c741a6a', 8, 2)
1500 ('air_0b184ec04c741a6a', 26, 2)
1600 ('air_0b1e72d2d4422b20', 25, 11)
1700 ('air_0e1eae99b8723bc1', 26, 12)
1800 ('air_0f0cdeee6c9bf3d7', 18, 11)
1900 ('air_1033310359ceeac1', 7, 4)
2000 ('air_1033310359ceeac1', 20, 10)
2100 ('air_12c4fb7a423df20d', 3, 12)
2200 ('air_12c4fb7a423df20d', 21, 2)
2300 ('air_1408dd53f31a8a65', 3, 12)
2400 ('air_1408dd53f31a8a65', 20, 11)
2500 ('air_142e78ba7001da9c', 7, 2)
2600 ('air_142e78ba7001da9c', 27, 3)
2700 ('ai

21900 ('air_a85f0c0c889f6b7e', 14, 3)
22000 ('air_a88ac559064dec08', 17, 3)
22100 ('air_a9178f19da58fe99', 8, 11)
22200 ('air_ab3ae0e410b20069', 6, 4)
22300 ('air_ade6e836ffd1da64', 1, 4)
22400 ('air_ade6e836ffd1da64', 19, 11)
22500 ('air_af63df35857b16e6', 6, 11)
22600 ('air_af63df35857b16e6', 26, 11)
22700 ('air_b2395df0e874078d', 8, 11)
22800 ('air_b23d0f519291247d', 11, 1)
22900 ('air_b23d0f519291247d', 29, 3)
23000 ('air_b2d97bd2337c5ba7', 10, 11)
23100 ('air_b439391e72899756', 26, 3)
23200 ('air_b45b8e456f53942a', 19, 3)
23300 ('air_b4f32bcc399da2b9', 10, 12)
23400 ('air_b4f32bcc399da2b9', 29, 10)
23500 ('air_b9e27558fb8bd5c4', 1, 2)
23600 ('air_b9e27558fb8bd5c4', 25, 2)
23700 ('air_bb09595bab7d5cfb', 13, 1)
23800 ('air_bb09595bab7d5cfb', 27, 1)
23900 ('air_bc991c51d6613745', 14, 3)
24000 ('air_bcce1ea4350b7b72', 10, 4)
24100 ('air_bebd55ed63ab2422', 31, 12)
24200 ('air_bedd35489e666605', 23, 5)
24300 ('air_bf617aa68d5f1cfa', 22, 11)
24400 ('air_bfafaed35e213fd7', 14, 1)
24500 ('

43500 ('hpg_0613d2d4b8dca3d8', 11, 12)
43600 ('hpg_0613d2d4b8dca3d8', 24, 3)
43700 ('hpg_0640338d8f68c004', 22, 12)
43800 ('hpg_06477715d4b11ecd', 20, 3)
43900 ('hpg_06497770e8ea7bcc', 28, 2)
44000 ('hpg_065b15947df4f68f', 15, 8)
44100 ('hpg_0669d5556455638a', 27, 3)
44200 ('hpg_067d53cddab62f07', 19, 1)
44300 ('hpg_06b06c43f6f6d985', 8, 8)
44400 ('hpg_06bff449883a674b', 7, 7)
44500 ('hpg_06c4d0cecd25c768', 8, 7)
44600 ('hpg_06c4d0cecd25c768', 24, 6)
44700 ('hpg_06e35f9c74f984c7', 18, 4)
44800 ('hpg_06e78a7f96ad28e5', 29, 12)
44900 ('hpg_06f666f89a1a15a0', 1, 4)
45000 ('hpg_06f666f89a1a15a0', 18, 6)
45100 ('hpg_070fec64f8bf53d6', 6, 3)
45200 ('hpg_070fec64f8bf53d6', 29, 10)
45300 ('hpg_071ca54758c7a759', 21, 12)
45400 ('hpg_07271a81a6e7b70a', 17, 2)
45500 ('hpg_07283eaa62dfb96c', 22, 7)
45600 ('hpg_072bdeb06f1e6d8b', 13, 1)
45700 ('hpg_072bdeb06f1e6d8b', 26, 8)
45800 ('hpg_072ced2ebef14284', 14, 1)
45900 ('hpg_072fbbf3b2a6a172', 4, 1)
46000 ('hpg_072fbbf3b2a6a172', 31, 12)
46100 ('hpg_

65100 ('hpg_11801306ce8bfb4f', 27, 8)
65200 ('hpg_119da6d0c11debdc', 3, 12)
65300 ('hpg_119da6d0c11debdc', 20, 8)
65400 ('hpg_11b188f4d57f59c6', 31, 3)
65500 ('hpg_11bd82696bb56282', 22, 7)
65600 ('hpg_11ea6dbef1d1ece3', 14, 12)
65700 ('hpg_11fd2a3f3246b8f2', 23, 3)
65800 ('hpg_1207423ae79ff82f', 8, 12)
65900 ('hpg_1207423ae79ff82f', 29, 5)
66000 ('hpg_122700d86df849f8', 2, 1)
66100 ('hpg_122700d86df849f8', 20, 10)
66200 ('hpg_122eeffc058f824a', 11, 1)
66300 ('hpg_122f093dd3f746a5', 2, 11)
66400 ('hpg_122f093dd3f746a5', 17, 8)
66500 ('hpg_1236bd7976a0a95d', 3, 6)
66600 ('hpg_1236bd7976a0a95d', 21, 12)
66700 ('hpg_123a171ad6c79a05', 7, 1)
66800 ('hpg_123a171ad6c79a05', 23, 3)
66900 ('hpg_123ee05ae210e32f', 21, 11)
67000 ('hpg_125d94c264be92ad', 28, 10)
67100 ('hpg_126f14a1fba90056', 8, 4)
67200 ('hpg_126f14a1fba90056', 21, 9)
67300 ('hpg_128bbb48531d80b0', 9, 12)
67400 ('hpg_1290ea97ea2e5c1a', 3, 8)
67500 ('hpg_12959054717b32e6', 4, 3)
67600 ('hpg_12959054717b32e6', 18, 7)
67700 ('hpg_1

86700 ('hpg_1b4fcd9d408b8693', 12, 12)
86800 ('hpg_1b4ffd57a3ca3805', 2, 5)
86900 ('hpg_1b5cd4ce90e656d1', 14, 12)
87000 ('hpg_1b5ff26a09a24ca2', 11, 2)
87100 ('hpg_1b79d1fe08590688', 4, 4)
87200 ('hpg_1b9c82559da3d2c8', 13, 2)
87300 ('hpg_1ba0142b1a86fa03', 15, 8)
87400 ('hpg_1bbffc19c0851366', 6, 2)
87500 ('hpg_1bbffc19c0851366', 22, 4)
87600 ('hpg_1be40ce052c57ee3', 16, 11)
87700 ('hpg_1bed6a2b3d6f9e38', 13, 5)
87800 ('hpg_1c0c1d77789767ca', 20, 2)
87900 ('hpg_1c2cc4e428361153', 17, 4)
88000 ('hpg_1c3e94363d82d5eb', 5, 4)
88100 ('hpg_1c3e94363d82d5eb', 26, 12)
88200 ('hpg_1c4a6b3264aeeabc', 29, 12)
88300 ('hpg_1c63b995899b069a', 16, 1)
88400 ('hpg_1c65e6289c0b78c6', 2, 5)
88500 ('hpg_1c65e6289c0b78c6', 13, 11)
88600 ('hpg_1c65e6289c0b78c6', 25, 6)
88700 ('hpg_1c6f1bdbba948006', 15, 8)
88800 ('hpg_1c8a77c452a361a6', 14, 2)
88900 ('hpg_1c90270983d9844c', 6, 2)
89000 ('hpg_1c90270983d9844c', 20, 9)
89100 ('hpg_1ca059dc34a171e4', 6, 7)
89200 ('hpg_1ca059dc34a171e4', 21, 12)
89300 ('hpg_

...

445500 ('hpg_cf725acea46e2b46', 13, 5)
445600 ('hpg_cf7f19a475bd9ede', 25, 3)
445700 ('hpg_cf87027aac8fce7c', 14, 11)
445800 ('hpg_cf89da7bfd34a7c6', 21, 4)
445900 ('hpg_cf8bce3e979ba481', 27, 2)
446000 ('hpg_cf91fb4d15a35542', 23, 3)
446100 ('hpg_cf94628ee3996c26', 3, 1)
446200 ('hpg_cf94628ee3996c26', 26, 12)
446300 ('hpg_cfb941c7ce92e619', 18, 3)
446400 ('hpg_cfe447af0df955e4', 10, 2)
446500 ('hpg_cfe4bbe158e113e6', 8, 3)
446600 ('hpg_cfe4bbe158e113e6', 26, 12)
446700 ('hpg_cfefeaa88ecabcd5', 26, 3)
446800 ('hpg_d009088e023cf53f', 13, 1)
446900 ('hpg_d009088e023cf53f', 25, 7)
447000 ('hpg_d0146ed5f0687bf3', 16, 9)
447100 ('hpg_d0150807837b25dd', 7, 5)
447200 ('hpg_d0150807837b25dd', 21, 4)
447300 ('hpg_d024dcce7f5b297e', 7, 4)
447400 ('hpg_d02653e1cec8034d', 14, 12)
447500 ('hpg_d02c6e8688cf2ecb', 4, 2)
447600 ('hpg_d02c6e8688cf2ecb', 28, 1)
447700 ('hpg_d02cbe571bd18214', 17, 4)
447800 ('hpg_d02cbe571bd18214', 31, 10)
447900 ('hpg_d03b635fe2044859', 2, 4)
448000 ('hpg_d0c88ff4b8d5d

466600 ('hpg_da40ff6fc4cb2c2d', 9, 9)
466700 ('hpg_da40ff6fc4cb2c2d', 30, 9)
466800 ('hpg_da52462e5578bf52', 23, 4)
466900 ('hpg_da646a90f9b67bbe', 17, 3)
467000 ('hpg_da70588dd88312d6', 14, 9)
467100 ('hpg_da865f190591fc76', 8, 4)
467200 ('hpg_da865f190591fc76', 29, 5)
467300 ('hpg_da8745bb98174c23', 26, 2)
467400 ('hpg_dab678a0aacc690d', 22, 1)
467500 ('hpg_dabd0715e31d68c3', 19, 3)
467600 ('hpg_dac4511cdccdbc3f', 16, 8)
467700 ('hpg_dac72789163a3f47', 7, 12)
467800 ('hpg_dac72789163a3f47', 20, 1)
467900 ('hpg_dacb0be112058dda', 3, 1)
468000 ('hpg_dacb0be112058dda', 26, 11)
468100 ('hpg_dacb598990b32bed', 16, 9)
468200 ('hpg_dad7855e06b14f4b', 4, 1)
468300 ('hpg_dad7855e06b14f4b', 24, 7)
468400 ('hpg_daeb1f59e784fdc0', 12, 8)
468500 ('hpg_daeb1f59e784fdc0', 30, 4)
468600 ('hpg_db0d375e7dc5b0a6', 4, 5)
468700 ('hpg_db0d375e7dc5b0a6', 26, 12)
468800 ('hpg_db3930bd5e1f3905', 19, 1)
468900 ('hpg_db3a86c77d2b7cbb', 14, 3)
469000 ('hpg_db47c13307a26561', 11, 3)
469100 ('hpg_db4a4341260a4e5

487700 ('hpg_e48aa7684524feaa', 20, 2)
487800 ('hpg_e48aa7684524feaa', 31, 12)
487900 ('hpg_e493dcc7be94497b', 25, 11)
488000 ('hpg_e4b61312f24166e2', 17, 3)
488100 ('hpg_e4ba73c8968f4ab9', 9, 3)
488200 ('hpg_e4ba73c8968f4ab9', 28, 3)
488300 ('hpg_e4c0b9eae8f94df1', 17, 10)
488400 ('hpg_e4d733fbe91d1a0f', 27, 1)
488500 ('hpg_e4f47311f9014ec9', 8, 10)
488600 ('hpg_e4f47311f9014ec9', 28, 11)
488700 ('hpg_e51a522e098f024c', 27, 2)
488800 ('hpg_e51e7a2f2bf9ce51', 27, 5)
488900 ('hpg_e52169bacb75561f', 2, 4)
489000 ('hpg_e52169bacb75561f', 29, 12)
489100 ('hpg_e535948c7db03ecf', 7, 3)
489200 ('hpg_e535948c7db03ecf', 24, 6)
489300 ('hpg_e545305bb007d534', 10, 4)
489400 ('hpg_e557e97ab993b5df', 6, 3)
489500 ('hpg_e562467a7233f227', 7, 2)
489600 ('hpg_e5633cb574f47afb', 24, 2)
489700 ('hpg_e56835cd19e8e5a7', 3, 2)
489800 ('hpg_e56835cd19e8e5a7', 19, 12)
489900 ('hpg_e56c26b81486482f', 12, 2)
490000 ('hpg_e5743d53d81e83a9', 7, 10)
490100 ('hpg_e5743d53d81e83a9', 29, 12)
490200 ('hpg_e57f119fb78

508700 ('hpg_ee80371461340fcb', 7, 9)
508800 ('hpg_ee8bd2cafed20951', 7, 12)
508900 ('hpg_ee8bd2cafed20951', 22, 11)
509000 ('hpg_ee93962415138f21', 11, 2)
509100 ('hpg_ee93962415138f21', 31, 12)
509200 ('hpg_eeb8ef13e476464f', 18, 11)
509300 ('hpg_eec1291c0483d586', 8, 9)
509400 ('hpg_eec1291c0483d586', 23, 2)
509500 ('hpg_eec829c8283386c8', 26, 3)
509600 ('hpg_eedb6e51e8d77a5d', 26, 2)
509700 ('hpg_eedc064525edce13', 21, 4)
509800 ('hpg_eedd8e8bfdd1d0f4', 18, 11)
509900 ('hpg_eee99d2a10c43381', 12, 3)
510000 ('hpg_ef073c02cdffe022', 10, 4)
510100 ('hpg_ef073c02cdffe022', 23, 3)
510200 ('hpg_ef15c1ef5bb1df12', 20, 5)
510300 ('hpg_ef16a4796e0d129f', 24, 8)
510400 ('hpg_ef2ad624c3cca506', 15, 3)
510500 ('hpg_ef3414d226142732', 22, 7)
510600 ('hpg_ef4354eede113561', 15, 11)
510700 ('hpg_ef63d8617a77b36a', 15, 4)
510800 ('hpg_ef781344cb4fa86c', 30, 12)
510900 ('hpg_ef81f4730fd23131', 16, 2)
511000 ('hpg_ef81f4730fd23131', 29, 9)
511100 ('hpg_ef8bea335ce46a6d', 21, 4)
511200 ('hpg_ef9d5078

529700 ('hpg_f82bd7b4153c76ab', 7, 10)
529800 ('hpg_f82bd7b4153c76ab', 30, 12)
529900 ('hpg_f8554590edd44845', 19, 3)
530000 ('hpg_f8702eb51dc5e68a', 8, 4)
530100 ('hpg_f8702eb51dc5e68a', 25, 4)
530200 ('hpg_f880ef4a3cd3dbaa', 24, 3)
530300 ('hpg_f8bb003c4c4870d7', 4, 3)
530400 ('hpg_f8bb003c4c4870d7', 26, 7)
530500 ('hpg_f8bdadbf50d02906', 27, 12)
530600 ('hpg_f8c41304612b6a98', 20, 4)
530700 ('hpg_f8c833663844d435', 15, 11)
530800 ('hpg_f8d132bb8882bb6f', 16, 10)
530900 ('hpg_f8dba580f220a068', 22, 4)
531000 ('hpg_f8f57280975dcd54', 8, 6)
531100 ('hpg_f8f57280975dcd54', 21, 7)
531200 ('hpg_f8fe1a6030e127ab', 4, 5)
531300 ('hpg_f8fe1a6030e127ab', 27, 5)
531400 ('hpg_f8ff1e1adb212f9f', 13, 3)
531500 ('hpg_f8ff1e1adb212f9f', 27, 1)
531600 ('hpg_f90f765f4bca7331', 13, 1)
531700 ('hpg_f92222c61d0adb11', 2, 9)
531800 ('hpg_f92222c61d0adb11', 19, 3)
531900 ('hpg_f929f85be803fe84', 8, 7)
532000 ('hpg_f929f85be803fe84', 22, 9)
532100 ('hpg_f9367411bcf6d574', 10, 11)
532200 ('hpg_f9367411bcf6d

* To check the effect of the day, the month and holiday on reservation

In [31]:
grouped=data.groupby(['holiday','day','month'])
data['holiday_day_month_index']=np.nan
for i,item in enumerate(grouped.groups):
    
    a=data.holiday==item[0]
    b=data.day==item[1]
    c=data.month==item[2]
    LOC=np.where(a & b & c)[0]
    data.loc[LOC,'holiday_day_month_index']=grouped.get_group(item).loc[:,'visitor'].mean()

* To check the effect of the year on reservation

In [28]:
grouped=data.groupby('year')
data['year_index']=np.nan
for i,item in enumerate(grouped.groups):
    loc_item=np.where(data['year']==item)[0]
    data.loc[loc_item,'year_index']=grouped.get_group(item).loc[:,'visitor'].mean()

* To check the effect of year and store on reservation

In [33]:
grouped=data.groupby(['store_id','year'])
data['store_year_index']=np.nan
for i,item in enumerate(grouped.groups):
    a=data.store_id==item[0]
    b=data.year==item[1]
    LOC=np.where(a & b)[0]
    if i%5000==0:print(i,item,datetime.now())
    data.loc[LOC,'store_year_index']=grouped.get_group(item).loc[:,'visitor'].mean()

0 ('air_00a91d42b08b08d9', 2016) 2018-04-28 16:55:45.011761
5000 ('hpg_80aaa8784a401d46', 2017) 2018-04-28 17:02:04.854946


* To check the effect of the date on reservation

In [29]:
grouped=data.groupby('date')
data['date_index']=np.nan
for i,item in enumerate(grouped.groups):
    loc_item=np.where(data['date']==item)[0]
    data.loc[loc_item,'date_index']=grouped.get_group(item).loc[:,'visitor'].mean()

In [37]:
print(data.describe())
print(data.head())

data.to_csv(os.path.join(output_data_path,'data1.csv'), encoding='utf-8',index=False)
data=pd.read_csv(os.path.join(output_data_path,'data1.csv'))

             visitor        holiday             dt      dayofweek  \
count  587957.000000  587957.000000  587957.000000  587957.000000   
mean        8.610766       0.064949     167.435631       3.361385   
std        10.973159       0.246435     190.366523       1.836051   
min         1.000000       0.000000       0.000000       0.000000   
25%         3.000000       0.000000      46.000000       2.000000   
50%         5.000000       0.000000     107.000000       4.000000   
75%        10.000000       0.000000     219.500000       5.000000   
max      2241.000000       1.000000    8429.284314       6.000000   

               month            day           year           hour  \
count  587957.000000  587957.000000  587957.000000  587957.000000   
mean        5.728307      16.341962    2016.344134      18.146951   
std         3.728686       8.653998       0.475085       2.135966   
min         1.000000       1.000000    2016.000000       0.000000   
25%         3.000000       9.0000

In [38]:
print(data.isnull().any())

id                               False
store_id                         False
genre                            False
visitor                          False
area                             False
holiday                          False
dt                               False
dayofweek                        False
month                            False
day                              False
year                             False
date                             False
hour                             False
store_index                      False
area_index                       False
genre_index                      False
genre_area_index                 False
holiday_index                    False
store_holiday_index              False
month_index                      False
store_month_index                False
dayofweek_index                  False
store_dayofweek_index            False
dayofweek_month_index            False
store_dayofweek_month_index      False
holiday_dayofweek_month_i